In [1]:
import pandas as pd
import numpy as np
import requests
import pyergastAPI
import datetime
import json
from tqdm.notebook import trange, tqdm

START_YEAR = 1990
END_YEAR = 2022

## Retrieve races

I'll use the simple yet effective library `pyergast`

In [2]:
races = pd.DataFrame()

for year in trange(START_YEAR, END_YEAR + 1):
    race = pyergastAPI.get_schedule(year)
    races = pd.concat([races, race])

print(races.shape)

  0%|          | 0/33 [00:00<?, ?it/s]

(595, 15)


In [3]:
races.head()

,season,round,url,raceName,date,circuitID,circuitName,locality,country,time,FirstPractice,SecondPractice,ThirdPractice,Qualifying,Sprint
0,1990,1,http://en.wikipedia.org/wiki/1990_United_State...,United States Grand Prix,1990-03-11,phoenix,Phoenix street circuit,Phoenix,USA,NaN,NaN,NaN,NaN,NaN,NaN
1,1990,2,http://en.wikipedia.org/wiki/1990_Brazilian_Gr...,Brazilian Grand Prix,1990-03-25,interlagos,Autódromo José Carlos Pace,São Paulo,Brazil,NaN,NaN,NaN,NaN,NaN,NaN
2,1990,3,http://en.wikipedia.org/wiki/1990_San_Marino_G...,San Marino Grand Prix,1990-05-13,imola,Autodromo Enzo e Dino Ferrari,Imola,Italy,NaN,NaN,NaN,NaN,NaN,NaN
3,1990,4,http://en.wikipedia.org/wiki/1990_Monaco_Grand...,Monaco Grand Prix,1990-05-27,monaco,Circuit de Monaco,Monte-Carlo,Monaco,NaN,NaN,NaN,NaN,NaN,NaN
4,1990,5,http://en.wikipedia.org/wiki/1990_Canadian_Gra...,Canadian Grand Prix,1990-06-10,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
races.tail()

,season,round,url,raceName,date,circuitID,circuitName,locality,country,time,FirstPractice,SecondPractice,ThirdPractice,Qualifying,Sprint
17,2022,18,http://en.wikipedia.org/wiki/2022_Japanese_Gra...,Japanese Grand Prix,2022-10-09,suzuka,Suzuka Circuit,Suzuka,Japan,05:00:00Z,"{'date': '2022-10-07', 'time': '04:00:00Z'}","{'date': '2022-10-07', 'time': '08:00:00Z'}","{'date': '2022-10-08', 'time': '04:00:00Z'}","{'date': '2022-10-08', 'time': '07:00:00Z'}",NaN
18,2022,19,http://en.wikipedia.org/wiki/2022_United_State...,United States Grand Prix,2022-10-23,americas,Circuit of the Americas,Austin,USA,19:00:00Z,"{'date': '2022-10-21', 'time': '19:00:00Z'}","{'date': '2022-10-21', 'time': '22:00:00Z'}","{'date': '2022-10-22', 'time': '19:00:00Z'}","{'date': '2022-10-22', 'time': '22:00:00Z'}",NaN
19,2022,20,http://en.wikipedia.org/wiki/2022_Mexican_Gran...,Mexico City Grand Prix,2022-10-30,rodriguez,Autódromo Hermanos Rodríguez,Mexico City,Mexico,20:00:00Z,"{'date': '2022-10-28', 'time': '18:00:00Z'}","{'date': '2022-10-28', 'time': '21:00:00Z'}","{'date': '2022-10-29', 'time': '17:00:00Z'}","{'date': '2022-10-29', 'time': '20:00:00Z'}",NaN
20,2022,21,http://en.wikipedia.org/wiki/2022_Brazilian_Gr...,Brazilian Grand Prix,2022-11-13,interlagos,Autódromo José Carlos Pace,São Paulo,Brazil,18:00:00Z,"{'date': '2022-11-11', 'time': '15:30:00Z'}","{'date': '2022-11-12', 'time': '15:30:00Z'}",NaN,"{'date': '2022-11-11', 'time': '19:00:00Z'}","{'date': '2022-11-12', 'time': '19:30:00Z'}"
21,2022,22,http://en.wikipedia.org/wiki/2022_Abu_Dhabi_Gr...,Abu Dhabi Grand Prix,2022-11-20,yas_marina,Yas Marina Circuit,Abu Dhabi,UAE,13:00:00Z,"{'date': '2022-11-18', 'time': '09:00:00Z'}","{'date': '2022-11-18', 'time': '12:00:00Z'}","{'date': '2022-11-19', 'time': '10:00:00Z'}","{'date': '2022-11-19', 'time': '13:00:00Z'}",NaN


In [5]:
# remove future races
races['date'] = pd.to_datetime(races['date'])
races = races.loc[races['date'] < datetime.datetime.now()]

In [6]:
# drop useless data
races.drop(['circuitName', 'locality', 'time', 'FirstPractice', 'SecondPractice', 'ThirdPractice', 'Qualifying', 'Sprint'], axis=1, inplace=True)

In [7]:
races.to_csv('data/races.csv', index=False)

## Race Results

In [8]:
# ergast returns a dict containing time in milli format and time + delta format, let's keep millis
def get_time(row: pd.DataFrame):
    if str(row) != 'nan':
        return row['millis']
    else:
        return row

In [9]:
races = pd.read_csv('data/races.csv')

results = pd.DataFrame()

for year in trange(START_YEAR, END_YEAR + 1):
    for round in races.loc[races['season'] == year]['round']:
        result = pyergastAPI.get_race_result(year, round)
        result['Time'] = result['Time'].apply(get_time)
        result['season'] = year
        result['round'] = round
        results = pd.concat([results, result])

  0%|          | 0/33 [00:00<?, ?it/s]

In [10]:
results.head()

,number,position,positionText,grid,points,driverID,driver,nationality,constructorID,constructor,laps,status,Time,season,round
0,27,1,1,5,9,senna,Ayrton Senna,Brazilian,mclaren,McLaren,72,Finished,6752829,1990,1
1,4,2,2,4,6,alesi,Jean Alesi,French,tyrrell,Tyrrell,72,Finished,6761514,1990,1
2,5,3,3,9,4,boutsen,Thierry Boutsen,Belgian,williams,Williams,72,Finished,6806909,1990,1
3,20,4,4,6,3,piquet,Nelson Piquet,Brazilian,benetton,Benetton,72,Finished,6821187,1990,1
4,8,5,5,10,2,modena,Stefano Modena,Italian,brabham,Brabham,72,Finished,6822332,1990,1


In [11]:
results.tail(10)

,number,position,positionText,grid,points,driverID,driver,nationality,constructorID,constructor,laps,status,Time,season,round
10,18,11,11,14,0,stroll,Lance Stroll,Canadian,aston_martin,Aston Martin,69,+1 Lap,NaN,2022,13
11,10,12,12,0,0,gasly,Pierre Gasly,French,alphatauri,AlphaTauri,69,+1 Lap,NaN,2022,13
12,24,13,13,12,0,zhou,Guanyu Zhou,Chinese,alfa,Alfa Romeo,69,+1 Lap,NaN,2022,13
13,47,14,14,15,0,mick_schumacher,Mick Schumacher,German,haas,Haas F1 Team,69,+1 Lap,NaN,2022,13
14,3,15,15,9,0,ricciardo,Daniel Ricciardo,Australian,mclaren,McLaren,69,+1 Lap,NaN,2022,13
15,20,16,16,13,0,kevin_magnussen,Kevin Magnussen,Danish,haas,Haas F1 Team,69,+1 Lap,NaN,2022,13
16,23,17,17,17,0,albon,Alexander Albon,Thai,williams,Williams,69,+1 Lap,NaN,2022,13
17,6,18,18,19,0,latifi,Nicholas Latifi,Canadian,williams,Williams,69,+1 Lap,NaN,2022,13
18,22,19,19,16,0,tsunoda,Yuki Tsunoda,Japanese,alphatauri,AlphaTauri,68,+2 Laps,NaN,2022,13
19,77,20,20,8,0,bottas,Valtteri Bottas,Finnish,alfa,Alfa Romeo,65,Power Unit,NaN,2022,13


In [12]:
results.to_csv('data/results.csv', index=False)

## Driver Standings

In [13]:
# I will use this function later to calculate points and wins prior to the race

def lookup (df, team, points):
    df['lookup1'] = df.season.astype(str) + df[team] + df['round'].astype(str)
    df['lookup2'] = df.season.astype(str) + df[team] + (df['round']-1).astype(str)
    new_df = df.merge(df[['lookup1', points]], how = 'left', left_on='lookup2',right_on='lookup1')
    new_df.drop(['lookup1_x', 'lookup2', 'lookup1_y'], axis = 1, inplace = True)
    new_df.rename(columns = {points+'_x': points+'_after_race', points+'_y': points}, inplace = True)
    new_df[points].fillna(0, inplace = True)
    return new_df

In [14]:
rounds = []
races = pd.read_csv('data/races.csv')
for year in np.array(races.season.unique()):
    rounds.append([year, list(races[races.season == year]['round'])])

In [15]:
driver_standings = {'season': [],
                    'round':[],
                    'driver': [],
                    'driver_points': [],
                    'driver_wins': [],
                   'driver_standings_pos': []}

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/driverStandings.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
            try:
                driver_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except:
                driver_standings['season'].append(None)

            try:
                driver_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except:
                driver_standings['round'].append(None)
                                         
            try:
                driver_standings['driver'].append(item['Driver']['driverId'])
            except:
                driver_standings['driver'].append(None)
            
            try:
                driver_standings['driver_points'].append(int(item['points']))
            except:
                driver_standings['driver_points'].append(None)
            
            try:
                driver_standings['driver_wins'].append(int(item['wins']))
            except:
                driver_standings['driver_wins'].append(None)
                
            try:
                driver_standings['driver_standings_pos'].append(int(item['position']))
            except:
                driver_standings['driver_standings_pos'].append(None)
            
driver_standings = pd.DataFrame(driver_standings)
print(driver_standings.shape)

(13455, 6)


In [16]:
driver_standings = lookup(driver_standings, 'driver', 'driver_points')
driver_standings = lookup(driver_standings, 'driver', 'driver_wins')
driver_standings = lookup(driver_standings, 'driver', 'driver_standings_pos')

In [17]:
driver_standings.head()

,season,round,driver,driver_points_after_race,driver_wins_after_race,driver_standings_pos_after_race,driver_points,driver_wins,driver_standings_pos
0,1990,1,senna,9.0,1,1,0.0,0.0,0.0
1,1990,1,alesi,6.0,0,2,0.0,0.0,0.0
2,1990,1,boutsen,4.0,0,3,0.0,0.0,0.0
3,1990,1,piquet,3.0,0,4,0.0,0.0,0.0
4,1990,1,modena,2.0,0,5,0.0,0.0,0.0


In [18]:
driver_standings.tail()

,season,round,driver,driver_points_after_race,driver_wins_after_race,driver_standings_pos_after_race,driver_points,driver_wins,driver_standings_pos
13450,2022,13,zhou,5.0,0,17,5.0,0.0,17.0
13451,2022,13,stroll,4.0,0,18,4.0,0.0,18.0
13452,2022,13,albon,3.0,0,19,3.0,0.0,19.0
13453,2022,13,latifi,0.0,0,20,0.0,0.0,20.0
13454,2022,13,hulkenberg,0.0,0,21,0.0,0.0,21.0


In [19]:
driver_standings.to_csv('data/driver_standings.csv', index = False)

## Constructor Standings

In [20]:
constructor_rounds = rounds#[8:]

constructor_standings = {'season': [],
                    'round':[],
                    'constructor': [],
                    'constructor_points': [],
                    'constructor_wins': [],
                   'constructor_standings_pos': []}

for n in list(range(len(constructor_rounds))):
    for i in constructor_rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/constructorStandings.json'
        r = requests.get(url.format(constructor_rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
            try:
                constructor_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except:
                constructor_standings['season'].append(None)

            try:
                constructor_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except:
                constructor_standings['round'].append(None)
                                         
            try:
                constructor_standings['constructor'].append(item['Constructor']['constructorId'])
            except:
                constructor_standings['constructor'].append(None)
            
            try:
                constructor_standings['constructor_points'].append(int(item['points']))
            except:
                constructor_standings['constructor_points'].append(None)
            
            try:
                constructor_standings['constructor_wins'].append(int(item['wins']))
            except:
                constructor_standings['constructor_wins'].append(None)
                
            try:
                constructor_standings['constructor_standings_pos'].append(int(item['position']))
            except:
                constructor_standings['constructor_standings_pos'].append(None)
            
constructor_standings = pd.DataFrame(constructor_standings)
print(constructor_standings.shape)

(6662, 6)


In [21]:
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_points')
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_wins')
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_standings_pos')

In [22]:
constructor_standings.head()

,season,round,constructor,constructor_points_after_race,constructor_wins_after_race,constructor_standings_pos_after_race,constructor_points,constructor_wins,constructor_standings_pos
0,1990,1,mclaren,9.0,1,1,0.0,0.0,0.0
1,1990,1,tyrrell,7.0,0,2,0.0,0.0,0.0
2,1990,1,williams,4.0,0,3,0.0,0.0,0.0
3,1990,1,benetton,3.0,0,4,0.0,0.0,0.0
4,1990,1,brabham,2.0,0,5,0.0,0.0,0.0


In [23]:
constructor_standings.tail()

,season,round,constructor,constructor_points_after_race,constructor_wins_after_race,constructor_standings_pos_after_race,constructor_points,constructor_wins,constructor_standings_pos
6657,2022,13,alfa,51.0,0,6,51.0,0.0,6.0
6658,2022,13,haas,34.0,0,7,34.0,0.0,7.0
6659,2022,13,alphatauri,27.0,0,8,27.0,0.0,8.0
6660,2022,13,aston_martin,20.0,0,9,19.0,0.0,9.0
6661,2022,13,williams,3.0,0,10,3.0,0.0,10.0


In [24]:
constructor_standings.to_csv('data/constructor_standings.csv', index=False)

## Weather

In [25]:
races = pd.read_csv('data/races.csv')
races.shape

(586, 7)

In [26]:
races.head()

,season,round,url,raceName,date,circuitID,country
0,1990,1,http://en.wikipedia.org/wiki/1990_United_State...,United States Grand Prix,1990-03-11,phoenix,USA
1,1990,2,http://en.wikipedia.org/wiki/1990_Brazilian_Gr...,Brazilian Grand Prix,1990-03-25,interlagos,Brazil
2,1990,3,http://en.wikipedia.org/wiki/1990_San_Marino_G...,San Marino Grand Prix,1990-05-13,imola,Italy
3,1990,4,http://en.wikipedia.org/wiki/1990_Monaco_Grand...,Monaco Grand Prix,1990-05-27,monaco,Monaco
4,1990,5,http://en.wikipedia.org/wiki/1990_Canadian_Gra...,Canadian Grand Prix,1990-06-10,villeneuve,Canada


In [27]:
def get_weather_from_url(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    #print(url)
    try:
        weather_row = soup.find('table', {'class': 'infobox vevent'}).find('th', text='Weather').find_next('td').text.strip()
    except AttributeError:
        italian_link = soup.find('span', text='Italiano').parent.get('href')
        print(italian_link)
        html = requests.get(italian_link).text
        soup = BeautifulSoup(html, 'html.parser')
        weather_row = soup.find('table', {'class': 'sinottico'}).find('th', text=lambda t: t and 'Clima' in t).find_next('td').text.strip()
    return weather_row

In [28]:
from bs4 import BeautifulSoup
tqdm.pandas(desc='Getting weather')

races['weather'] = races['url'].progress_apply(lambda row: get_weather_from_url(row))
races.head()

Getting weather:   0%|          | 0/586 [00:00<?, ?it/s]

https://it.wikipedia.org/wiki/Gran_Premio_del_Portogallo_1990
https://it.wikipedia.org/wiki/Gran_Premio_di_Francia_1991
https://it.wikipedia.org/wiki/Gran_Premio_di_Francia_1993
https://it.wikipedia.org/wiki/Gran_Premio_d%27Ungheria_1993
https://it.wikipedia.org/wiki/Gran_Premio_d%27Europa_2006
https://it.wikipedia.org/wiki/Gran_Premio_di_Cina_2013
https://it.wikipedia.org/wiki/Gran_Premio_del_Bahrein_2013
https://it.wikipedia.org/wiki/Gran_Premio_del_Belgio_2013
https://it.wikipedia.org/wiki/Gran_Premio_del_Bahrein_2017
https://it.wikipedia.org/wiki/Gran_Premio_di_Russia_2017
https://it.wikipedia.org/wiki/Gran_Premio_di_Spagna_2017
https://it.wikipedia.org/wiki/Gran_Premio_di_Monaco_2017
https://it.wikipedia.org/wiki/Gran_Premio_del_Brasile_2017
https://it.wikipedia.org/wiki/Gran_Premio_del_Bahrein_2018
https://it.wikipedia.org/wiki/Gran_Premio_di_Cina_2018
https://it.wikipedia.org/wiki/Gran_Premio_di_Monaco_2018
https://it.wikipedia.org/wiki/Gran_Premio_d%27Ungheria_2018
https://it.w

,season,round,url,raceName,date,circuitID,country,weather
0,1990,1,http://en.wikipedia.org/wiki/1990_United_State...,United States Grand Prix,1990-03-11,phoenix,USA,Cloudy and cool with temperatures reaching up ...
1,1990,2,http://en.wikipedia.org/wiki/1990_Brazilian_Gr...,Brazilian Grand Prix,1990-03-25,interlagos,Brazil,"Hot, dry, sunny"
2,1990,3,http://en.wikipedia.org/wiki/1990_San_Marino_G...,San Marino Grand Prix,1990-05-13,imola,Italy,"Warm, dry, sunny"
3,1990,4,http://en.wikipedia.org/wiki/1990_Monaco_Grand...,Monaco Grand Prix,1990-05-27,monaco,Monaco,"Warm, dry, sunny"
4,1990,5,http://en.wikipedia.org/wiki/1990_Canadian_Gra...,Canadian Grand Prix,1990-06-10,villeneuve,Canada,Warm and cloudy with temperatures approaching ...


In [29]:
weather_dict = {'weather_warm': ['soleggiato', 'clear', 'warm', 'hot', 'sunny', 'fine', 'mild', 'sereno'],
               'weather_cold': ['cold', 'fresh', 'chilly', 'cool'],
               'weather_dry': ['dry', 'asciutto'],
               'weather_wet': ['showers', 'wet', 'rain', 'pioggia', 'damp', 'thunderstorms', 'rainy'],
               'weather_cloudy': ['overcast', 'nuvoloso', 'clouds', 'cloudy', 'grey', 'coperto']}

weather_df = pd.DataFrame(columns = weather_dict.keys())
for col in weather_df:
    weather_df[col] = races['weather'].map(lambda x: 1 if any(i in weather_dict[col] for i in x.lower().split()) else 0)

weather_df.head()

,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,0,1,0,0,1
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,1


In [30]:
weather_info = pd.concat([races, weather_df], axis=1)
weather_info.shape

(586, 13)

In [31]:
weather_info.head()

,season,round,url,raceName,date,circuitID,country,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,1990,1,http://en.wikipedia.org/wiki/1990_United_State...,United States Grand Prix,1990-03-11,phoenix,USA,Cloudy and cool with temperatures reaching up ...,0,1,0,0,1
1,1990,2,http://en.wikipedia.org/wiki/1990_Brazilian_Gr...,Brazilian Grand Prix,1990-03-25,interlagos,Brazil,"Hot, dry, sunny",1,0,0,0,0
2,1990,3,http://en.wikipedia.org/wiki/1990_San_Marino_G...,San Marino Grand Prix,1990-05-13,imola,Italy,"Warm, dry, sunny",1,0,0,0,0
3,1990,4,http://en.wikipedia.org/wiki/1990_Monaco_Grand...,Monaco Grand Prix,1990-05-27,monaco,Monaco,"Warm, dry, sunny",1,0,0,0,0
4,1990,5,http://en.wikipedia.org/wiki/1990_Canadian_Gra...,Canadian Grand Prix,1990-06-10,villeneuve,Canada,Warm and cloudy with temperatures approaching ...,1,0,0,0,1


In [32]:
weather_info.drop(['url', 'raceName', 'date', 'country'], axis=1, inplace=True)
weather_info.head()

,season,round,circuitID,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,1990,1,phoenix,Cloudy and cool with temperatures reaching up ...,0,1,0,0,1
1,1990,2,interlagos,"Hot, dry, sunny",1,0,0,0,0
2,1990,3,imola,"Warm, dry, sunny",1,0,0,0,0
3,1990,4,monaco,"Warm, dry, sunny",1,0,0,0,0
4,1990,5,villeneuve,Warm and cloudy with temperatures approaching ...,1,0,0,0,1


In [33]:
weather_info.to_csv('data/weather.csv', index=False)